In [1]:
%run ml_header.py

In [6]:
es_v4 = pd.read_csv('exhaustive_training_v4.csv')
es_v4_ada = pd.read_csv('exhaustive_training_v4_ada.csv')
es_v4_rf = pd.read_csv('exhaustive_training_v4_rf.csv')

# direct mapping
dm_v4 = pd.read_csv('direct_mapping_v4.csv')
dm_v5 = pd.read_csv('direct_mapping_v5.csv')

# direct mapping with tolerance
dm_v4_01 = pd.read_csv('direct_mapping_v4_tolerance_0.1.csv')
dm_v5_01 = pd.read_csv('direct_mapping_v5_tolerance_0.1.csv')
dm_v4_005 = pd.read_csv('direct_mapping_v4_tolerance_0.05.csv')
dm_v5_005 = pd.read_csv('direct_mapping_v5_tolerance_0.05.csv')

In [327]:
es_v4.head()

,train,test,adrs_ave,adrs_max,adrs_ave_rms,adrs_max_rms,hypervolume,dominance,cardinality
0,adpcm_encoder,aes,0.9112,2.3923,0.8708,2.2586,0.3165,0.4000,3.0
1,adpcm_encoder,ann,0.0389,0.0981,0.1739,0.7444,0.6894,0.3333,7.0
2,adpcm_encoder,average,0.0000,0.0000,0.0000,0.0000,0.4315,1.0000,3.0
3,adpcm_encoder,decimation,0.0100,0.0551,0.0162,0.0572,0.9891,0.7143,7.0
4,adpcm_encoder,fft_fixed,0.1176,0.1667,0.0381,0.0685,0.7579,0.0000,9.0


In [7]:
es_v4_rf.head()

,train,test,adrs_ave,adrs_max,adrs_ave_rms,adrs_max_rms,hypervolume,dominance,cardinality
0,adpcm_encoder,aes,0.0644,0.1712,0.1314,0.3564,0.8599,0.4000,3.0
1,adpcm_encoder,ann,0.0389,0.0981,0.1767,0.7472,0.9604,0.3333,5.0
2,adpcm_encoder,average,0.0000,0.0000,0.0000,0.0000,1.0000,1.0000,1.0
3,adpcm_encoder,decimation,0.0314,0.1415,0.0333,0.0624,0.9852,0.4286,4.0
4,adpcm_encoder,fft_fixed,0.0525,0.1267,0.0526,0.1268,0.9392,0.3333,11.0


Show the benchmarks, which form the training set, leading better results than direct mapping on the rest testing benchmarks.

In [8]:
es_v4 = es_v4_rf.copy()
train_combos = set(es_v4['train'])
for combo in train_combos:
    combo_list = combo.split(';')
    metrics_1 = ['adrs_ave']
    metrics_2 = ['hypervolume', 'dominance']
    bool_1 = es_v4[es_v4['train'] == combo][metrics_1].reset_index(drop=True) <= \
             dm_v4[~dm_v4['benchmark'].isin(combo_list)][metrics_1].reset_index(drop=True)
    bool_2 = es_v4[es_v4['train'] == combo][metrics_2].reset_index(drop=True) >= \
             dm_v4[~dm_v4['benchmark'].isin(combo_list)][metrics_2].reset_index(drop=True)
    bool_all = pd.concat([bool_1, bool_2], axis=1)
    if np.all(bool_all):
        print(combo)

adpcm_encoder;aes;ann;average;fir;idct;sobel
adpcm_encoder;aes;ann;fft_fixed;idct;interpolation;kasumi;qsort;snow3g;sobel
adpcm_encoder;aes;ann;decimation;fir;idct;kasumi;qsort
adpcm_encoder;aes;ann;fft_fixed;fir;idct;interpolation;qsort;snow3g;sobel
adpcm_encoder;aes;ann;average;decimation;fft_fixed;fir;idct;interpolation;kasumi;qsort;snow3g
adpcm_encoder;aes;ann;decimation;fft_fixed;fir;qsort;snow3g
adpcm_encoder;aes;ann;average;decimation;fir;idct;interpolation;kasumi;qsort;snow3g
adpcm_encoder;aes;ann;fft_fixed;kasumi;qsort;snow3g;sobel
adpcm_encoder;aes;ann;average;decimation;idct;interpolation;kasumi;qsort;snow3g;sobel
adpcm_encoder;aes;ann;fft_fixed;fir;interpolation;kasumi;qsort;snow3g
adpcm_encoder;aes;ann;average;decimation;fir;kasumi;qsort;snow3g
adpcm_encoder;aes;ann;idct;kasumi;qsort;snow3g;sobel
adpcm_encoder;aes;ann;fir;idct;sobel
adpcm_encoder;aes;ann;average;decimation;fft_fixed;fir;idct;interpolation;qsort;snow3g;sobel
adpcm_encoder;aes;ann;average;fft_fixed;fir;idct;

In [9]:
es_v4[es_v4['train'] == 'adpcm_encoder;aes;ann;fir;idct;sobel']

,train,test,adrs_ave,adrs_max,adrs_ave_rms,adrs_max_rms,hypervolume,dominance,cardinality
21228,adpcm_encoder;aes;ann;fir;idct;sobel,average,0.0000,0.0000,0.0000,0.0000,1.0000,1.0000,1.0
21229,adpcm_encoder;aes;ann;fir;idct;sobel,decimation,0.0374,0.1482,0.0373,0.1479,0.8898,0.5714,6.0
21230,adpcm_encoder;aes;ann;fir;idct;sobel,fft_fixed,0.0435,0.0993,0.0417,0.0994,0.9443,0.1667,8.0
21231,adpcm_encoder;aes;ann;fir;idct;sobel,interpolation,0.0505,0.2385,0.0247,0.0571,0.8175,0.2857,16.0
21232,adpcm_encoder;aes;ann;fir;idct;sobel,kasumi,0.1375,0.2559,0.1399,0.2621,0.7833,0.2857,4.0
21233,adpcm_encoder;aes;ann;fir;idct;sobel,qsort,0.0009,0.0063,0.0010,0.0071,0.9763,0.8571,10.0
21234,adpcm_encoder;aes;ann;fir;idct;sobel,snow3g,0.1039,0.2078,0.1040,0.2080,0.7188,0.5000,5.0


Show the cases when some benchmarks as training set leading to better results on some other testing benchmarks.

In [320]:
train_combos = set(es_v4['train'])
for combo in train_combos:
    combo_list = combo.split(';')
    metrics_1 = ['adrs_ave']
    metrics_2 = ['hypervolume', 'dominance']
    bool_1 = es_v4[es_v4['train'] == combo][metrics_1].reset_index(drop=True) <= \
             dm_v4[~dm_v4['benchmark'].isin(combo_list)][metrics_1].reset_index(drop=True)
    bool_2 = es_v4[es_v4['train'] == combo][metrics_2].reset_index(drop=True) >= \
             dm_v4[~dm_v4['benchmark'].isin(combo_list)][metrics_2].reset_index(drop=True)
    bool_all = pd.concat([bool_1, bool_2], axis=1)
    valid_bench = list(es_v4[es_v4['train'] == combo].reset_index(drop=True)[np.all(bool_all, axis=1)]['test'])
    if len(valid_bench) > 0:
        print(combo, valid_bench)

adpcm_encoder;average;decimation;fft_fixed;fir;snow3g ['aes', 'idct', 'kasumi']
adpcm_encoder;ann;average;decimation;idct;interpolation ['fft_fixed']
idct;interpolation;kasumi;qsort ['decimation', 'sobel']
adpcm_encoder;aes;ann;average;decimation;fft_fixed;idct;sobel ['fir', 'interpolation']
adpcm_encoder;fir;idct;qsort ['average', 'decimation', 'sobel']
ann;fft_fixed;idct;kasumi;snow3g ['average', 'decimation', 'fir', 'interpolation']
aes;ann;average;decimation;fir;idct;interpolation;kasumi;sobel ['snow3g']
aes;average;decimation;idct;interpolation;kasumi ['snow3g']
adpcm_encoder;aes;fft_fixed;fir;qsort;snow3g ['average', 'decimation', 'idct', 'kasumi']
aes;decimation;fft_fixed;idct;qsort;snow3g ['average', 'kasumi', 'sobel']
aes;decimation;kasumi;qsort;sobel ['idct']
adpcm_encoder;ann;fir;interpolation;kasumi;qsort ['average']
adpcm_encoder;average;decimation;kasumi;snow3g ['idct']
aes;decimation;fir;idct;kasumi;sobel ['average', 'interpolation', 'snow3g']
adpcm_encoder;average;decim

ann;decimation;fft_fixed;idct;interpolation;qsort;sobel ['fir']
ann;average;decimation;fir;qsort ['idct']
aes;ann;average;decimation;fir;idct;interpolation;sobel ['fft_fixed']
aes;ann;average;decimation;idct;kasumi;sobel ['snow3g']
fir;interpolation;kasumi;snow3g ['average']
ann;average;fft_fixed;fir;interpolation;qsort ['decimation', 'idct', 'snow3g']
ann;average;decimation;fir;idct;interpolation;kasumi;sobel ['snow3g']
adpcm_encoder;average;kasumi;qsort ['decimation', 'fir', 'idct']
adpcm_encoder;ann;idct;qsort ['fir']
adpcm_encoder;aes;ann;decimation;fft_fixed;fir;kasumi;qsort;snow3g;sobel ['idct']
adpcm_encoder;aes;decimation;interpolation;kasumi;qsort;snow3g ['average', 'fir']
fft_fixed;fir;kasumi;qsort ['adpcm_encoder', 'aes', 'ann', 'average', 'decimation', 'sobel']
adpcm_encoder;average;qsort;snow3g ['aes', 'fft_fixed', 'idct', 'kasumi']
average;decimation;fft_fixed;idct;interpolation;kasumi ['fir', 'snow3g', 'sobel']
adpcm_encoder;ann;average;fir;kasumi;qsort;snow3g ['idct']
a

decimation;fft_fixed;idct;qsort;snow3g ['average', 'sobel']
aes;fir;idct;kasumi;qsort;snow3g;sobel ['average', 'decimation']
aes;decimation;fft_fixed;fir;interpolation;sobel ['average', 'idct']
aes;ann;average;decimation;kasumi;snow3g ['sobel']
adpcm_encoder;aes;ann;fft_fixed;fir;kasumi;snow3g ['average', 'idct']
adpcm_encoder;decimation;idct;interpolation;qsort ['average', 'fft_fixed']
ann;average;decimation;idct;sobel ['fft_fixed']
aes;ann;decimation;fft_fixed;interpolation;snow3g;sobel ['average']
aes;ann;average;decimation;fft_fixed;fir;interpolation;snow3g;sobel ['idct']
aes;ann;average;decimation;fir;kasumi;snow3g ['idct']
adpcm_encoder;aes;ann;average;decimation;fir;kasumi;snow3g;sobel ['idct']
average;fir;interpolation;kasumi;qsort;snow3g;sobel ['decimation']
adpcm_encoder;aes;ann;fft_fixed;idct;interpolation;kasumi;snow3g;sobel ['decimation', 'fir']
adpcm_encoder;aes;ann;average;fft_fixed;idct;interpolation;qsort;snow3g;sobel ['decimation', 'fir']
aes;average;fft_fixed;fir;sob

average;fft_fixed;idct;qsort;snow3g ['decimation', 'sobel']
adpcm_encoder;fir;kasumi ['decimation', 'idct']
aes;decimation;fir;idct;interpolation;kasumi;sobel ['average', 'snow3g']
aes;average;fft_fixed;fir;interpolation;qsort;snow3g;sobel ['decimation', 'idct']
aes;average;decimation;fft_fixed;fir;idct;interpolation;kasumi;snow3g ['sobel']
aes;ann;decimation;fir;kasumi ['average']
adpcm_encoder;aes;average;decimation;fir;interpolation ['sobel']
average;fir;idct;interpolation;kasumi;qsort;snow3g ['decimation']
ann;average;fft_fixed;fir;idct;qsort;snow3g;sobel ['decimation', 'interpolation']
aes;fir;idct;interpolation;qsort;sobel ['average', 'decimation', 'snow3g']
adpcm_encoder;ann;fft_fixed;idct;interpolation ['fir']
aes;decimation;fft_fixed;fir;interpolation;kasumi;qsort;snow3g;sobel ['average', 'idct']
adpcm_encoder;ann;fft_fixed;fir;idct;interpolation;sobel ['decimation']
aes;ann;decimation;fft_fixed;fir;interpolation;kasumi;qsort ['idct']
adpcm_encoder;aes;ann;average;fft_fixed;id

aes;decimation;interpolation;qsort ['average', 'idct', 'snow3g', 'sobel']
average;decimation;kasumi;qsort;snow3g;sobel ['idct']
ann;average;snow3g ['decimation', 'fir']
adpcm_encoder;aes;average;idct;kasumi;qsort ['decimation', 'sobel']
adpcm_encoder;aes;ann;fft_fixed;idct;kasumi;qsort;snow3g;sobel ['decimation', 'fir', 'interpolation']
ann;fft_fixed;fir;idct;interpolation;snow3g ['decimation']
aes;average;interpolation;snow3g ['fir']
adpcm_encoder;ann;average;decimation;interpolation;qsort;snow3g;sobel ['fir']
aes;average;fir;qsort;snow3g ['idct', 'kasumi']
fft_fixed;idct;kasumi;sobel ['average', 'decimation']
aes;ann;average;decimation;interpolation;snow3g ['fir']
decimation;fft_fixed;idct;qsort;snow3g;sobel ['average', 'kasumi']
aes;average;fir;idct;kasumi;qsort;snow3g ['decimation', 'sobel']
ann;decimation;fft_fixed;kasumi;qsort ['fir', 'idct']
average;decimation;idct;kasumi;qsort ['sobel']
adpcm_encoder;ann;idct;interpolation;kasumi;qsort;snow3g;sobel ['fir']
aes;ann;average;fft_f

...

adpcm_encoder;ann;average;snow3g;sobel ['fir', 'idct']
ann;average;idct;kasumi;qsort;sobel ['snow3g']
aes;decimation;fft_fixed;interpolation;kasumi;qsort;snow3g;sobel ['average']
adpcm_encoder;ann;average;fft_fixed;idct;kasumi;qsort;snow3g;sobel ['decimation', 'fir', 'interpolation']
adpcm_encoder;average;fir;idct;interpolation;kasumi;qsort ['decimation', 'snow3g']
aes;ann;decimation;fft_fixed;idct;kasumi;qsort;snow3g ['average', 'fir', 'interpolation']
adpcm_encoder;ann;decimation;fir;idct;kasumi;qsort;snow3g ['average']
aes;ann;decimation;fft_fixed;idct;interpolation;qsort;snow3g ['fir']
aes;ann;fft_fixed;kasumi;sobel ['fir', 'idct']
adpcm_encoder;ann;fft_fixed;interpolation;snow3g;sobel ['average']
adpcm_encoder;ann;average;decimation;idct;interpolation;kasumi;qsort;snow3g;sobel ['fir']
aes;ann;decimation;fft_fixed;fir;kasumi;qsort ['idct']
adpcm_encoder;aes;average;decimation;fir;idct;kasumi;qsort ['sobel']
adpcm_encoder;decimation;fft_fixed;fir;idct ['average', 'kasumi', 'snow3g',

adpcm_encoder;aes;average;decimation;fft_fixed;fir;interpolation;qsort;sobel ['idct']
ann;average;decimation;fft_fixed;interpolation;kasumi;qsort ['fir', 'idct']
adpcm_encoder;aes;fft_fixed;idct;interpolation;kasumi;qsort;snow3g ['average', 'decimation', 'fir', 'sobel']
decimation;fft_fixed;kasumi;sobel ['adpcm_encoder', 'aes', 'ann', 'average', 'fir']
aes;fir;idct;kasumi;qsort;sobel ['average', 'decimation']
decimation;fft_fixed;fir;idct;interpolation;kasumi;qsort;snow3g ['average']
ann;average;decimation;fft_fixed;idct;interpolation;kasumi;sobel ['fir', 'snow3g']
aes;interpolation;qsort ['average', 'decimation', 'idct', 'snow3g', 'sobel']
adpcm_encoder;ann;average;decimation;fft_fixed;interpolation;kasumi;qsort ['fir', 'idct']
decimation;idct;qsort;snow3g ['average', 'sobel']
average;fft_fixed;fir;idct;interpolation;qsort ['decimation', 'snow3g', 'sobel']
adpcm_encoder;average;decimation;fir;kasumi;qsort;snow3g ['idct']
adpcm_encoder;ann;decimation;fft_fixed;fir;idct;sobel ['interpol

adpcm_encoder;aes;decimation;fir;kasumi;qsort;snow3g ['idct']
aes;decimation;fft_fixed;idct;kasumi;qsort;snow3g;sobel ['average']
average;decimation;fir;idct;snow3g ['interpolation', 'kasumi', 'sobel']
aes;ann;average;fft_fixed;kasumi;qsort;snow3g ['fir', 'idct']
adpcm_encoder;average;decimation;kasumi;qsort;snow3g;sobel ['idct']
adpcm_encoder;average;decimation;fft_fixed;idct;interpolation;kasumi ['fir', 'snow3g']
ann;fir;kasumi;snow3g ['average', 'decimation']
ann;average;decimation;interpolation;kasumi;qsort ['fir']
aes;ann;fft_fixed;qsort;snow3g;sobel ['fir', 'idct']
adpcm_encoder;average;fft_fixed;idct;kasumi;snow3g ['decimation', 'sobel']
ann;decimation;fft_fixed;idct;interpolation;snow3g ['fir']
aes;ann;decimation ['interpolation']
aes;ann;fft_fixed;idct;kasumi;qsort ['decimation', 'fir', 'interpolation', 'snow3g']
adpcm_encoder;ann;fir;kasumi;snow3g ['average', 'idct']
adpcm_encoder;ann;average;decimation;fft_fixed;idct;kasumi;qsort;snow3g;sobel ['fir', 'interpolation']
ann;ave

aes;ann;fir;kasumi;sobel ['average', 'decimation']
decimation;idct;interpolation;kasumi;sobel ['average', 'snow3g']
aes;ann;fft_fixed;idct;interpolation;kasumi;sobel ['decimation']
decimation;fft_fixed;fir;idct;qsort ['average', 'kasumi', 'sobel']
adpcm_encoder;aes;ann;average;fir;idct;interpolation;kasumi ['snow3g']
adpcm_encoder;idct;kasumi;qsort;snow3g ['decimation', 'sobel']
decimation;fir;idct;interpolation;snow3g;sobel ['average']
decimation;kasumi;snow3g;sobel ['idct']
adpcm_encoder;ann;average;fft_fixed;fir;idct;interpolation;qsort;snow3g ['decimation']
adpcm_encoder;aes;ann;average;fft_fixed;fir;interpolation;qsort ['decimation', 'sobel']
aes;ann;fft_fixed;kasumi;qsort ['fir', 'idct']
aes;average;fft_fixed;interpolation;qsort;sobel ['decimation', 'fir', 'idct']
adpcm_encoder;aes;fft_fixed;fir;idct;interpolation;kasumi;qsort;snow3g ['average', 'decimation', 'sobel']
adpcm_encoder;ann;fft_fixed;snow3g ['average', 'fir', 'idct']
average;fir;interpolation;kasumi;qsort;sobel ['deci

adpcm_encoder;fft_fixed;fir;interpolation;snow3g;sobel ['decimation']
adpcm_encoder;average;decimation;fft_fixed;fir;kasumi;qsort ['aes', 'ann', 'idct']
ann;decimation;fft_fixed;fir;idct;interpolation;kasumi;sobel ['snow3g']
aes;fir;interpolation;sobel ['average', 'decimation']
ann;decimation;fft_fixed;fir;kasumi;sobel ['idct']
adpcm_encoder;average;fft_fixed;kasumi;sobel ['aes', 'decimation', 'fir', 'idct']
average;fir;kasumi;qsort;sobel ['idct']
adpcm_encoder;ann;decimation;fft_fixed;fir;interpolation;qsort;sobel ['idct']
decimation;fir;idct;interpolation ['average', 'fft_fixed']
ann;average;decimation;fft_fixed;fir;qsort;snow3g;sobel ['idct']
adpcm_encoder;ann;decimation;fir;idct;kasumi;snow3g;sobel ['average']
adpcm_encoder;aes;average;fir;interpolation;sobel ['decimation']
aes;decimation;idct;qsort ['average', 'sobel']
ann;average;decimation;fft_fixed;idct;interpolation;kasumi;qsort;snow3g ['sobel']
aes;ann;average;fft_fixed;fir;idct;qsort ['decimation', 'interpolation', 'snow3g']

Show the benchmarks, which form the training set and using AdaBoost Decision Tree, leading better results than direct mapping on the rest testing benchmarks.

In [325]:
train_combos = set(es_v4_ada['train'])
for combo in train_combos:
    combo_list = combo.split(';')
    metrics_1 = ['adrs_ave']
    metrics_2 = ['hypervolume', 'dominance']
    bool_1 = es_v4_ada[es_v4_ada['train'] == combo][metrics_1].reset_index(drop=True) <= \
             dm_v4[~dm_v4['benchmark'].isin(combo_list)][metrics_1].reset_index(drop=True)
    bool_2 = es_v4_ada[es_v4_ada['train'] == combo][metrics_2].reset_index(drop=True) >= \
             dm_v4[~dm_v4['benchmark'].isin(combo_list)][metrics_2].reset_index(drop=True)
    bool_all = pd.concat([bool_1, bool_2], axis=1)
    if np.all(bool_all):
        print(combo)

adpcm_encoder;aes;ann;average;decimation;fft_fixed;fir;interpolation;kasumi;qsort;snow3g;sobel
adpcm_encoder;aes;ann;fft_fixed;fir;idct;interpolation;kasumi;qsort;snow3g;sobel
adpcm_encoder;aes;ann;average;fft_fixed;idct;kasumi;qsort;snow3g;sobel
adpcm_encoder;aes;ann;average;fft_fixed;fir;idct;interpolation;kasumi;qsort;snow3g;sobel
adpcm_encoder;aes;ann;decimation;fft_fixed;idct;interpolation;kasumi;qsort;snow3g;sobel
adpcm_encoder;aes;ann;average;decimation;fft_fixed;fir;idct;kasumi;qsort;sobel
adpcm_encoder;aes;ann;average;fft_fixed;idct;interpolation;kasumi;qsort;sobel
adpcm_encoder;aes;ann;average;fft_fixed;idct;interpolation;kasumi;qsort;snow3g;sobel
adpcm_encoder;aes;ann;decimation;fft_fixed;fir;idct;interpolation;kasumi;qsort;snow3g;sobel
adpcm_encoder;aes;ann;fft_fixed;idct;interpolation;kasumi;qsort;snow3g;sobel
adpcm_encoder;aes;ann;average;decimation;fft_fixed;fir;idct;interpolation;kasumi;qsort;sobel
adpcm_encoder;aes;ann;average;decimation;fft_fixed;idct;kasumi;qsort;sob

In [326]:
train_combos = set(es_v4_ada['train'])
for combo in train_combos:
    combo_list = combo.split(';')
    metrics_1 = ['adrs_ave']
    metrics_2 = ['hypervolume', 'dominance']
    bool_1 = es_v4_ada[es_v4_ada['train'] == combo][metrics_1].reset_index(drop=True) <= \
             dm_v4[~dm_v4['benchmark'].isin(combo_list)][metrics_1].reset_index(drop=True)
    bool_2 = es_v4_ada[es_v4_ada['train'] == combo][metrics_2].reset_index(drop=True) >= \
             dm_v4[~dm_v4['benchmark'].isin(combo_list)][metrics_2].reset_index(drop=True)
    bool_all = pd.concat([bool_1, bool_2], axis=1)
    valid_bench = list(es_v4_ada[es_v4_ada['train'] == combo].reset_index(drop=True)[np.all(bool_all, axis=1)]['test'])
    if len(valid_bench) > 0:
        print(combo, valid_bench)

adpcm_encoder;average;decimation;fft_fixed;fir;snow3g ['aes', 'idct', 'kasumi']
adpcm_encoder;ann;average;decimation;idct;interpolation ['fft_fixed']
idct;interpolation;kasumi;qsort ['decimation', 'sobel']
adpcm_encoder;aes;ann;average;decimation;fft_fixed;idct;sobel ['fir', 'interpolation']
adpcm_encoder;fir;idct;qsort ['average', 'decimation', 'sobel']
ann;fft_fixed;idct;kasumi;snow3g ['average', 'decimation', 'fir', 'interpolation']
aes;ann;average;decimation;fir;idct;interpolation;kasumi;sobel ['snow3g']
aes;average;decimation;idct;interpolation;kasumi ['snow3g']
adpcm_encoder;aes;fft_fixed;fir;qsort;snow3g ['average', 'decimation', 'idct', 'kasumi']
aes;decimation;fft_fixed;idct;qsort;snow3g ['average', 'kasumi', 'sobel']
aes;decimation;kasumi;qsort;sobel ['idct']
adpcm_encoder;ann;fir;interpolation;kasumi;qsort ['average']
adpcm_encoder;average;decimation;kasumi;snow3g ['idct']
aes;decimation;fir;idct;kasumi;sobel ['average', 'interpolation', 'snow3g']
adpcm_encoder;average;decim

aes;ann;average;decimation;fft_fixed;idct;kasumi;sobel ['fir', 'interpolation', 'snow3g']
fft_fixed;idct;interpolation;snow3g ['average', 'decimation', 'fir', 'kasumi', 'sobel']
ann;decimation;fft_fixed;idct;interpolation;qsort;sobel ['fir']
ann;average;decimation;fir;qsort ['idct']
aes;ann;average;decimation;fir;idct;interpolation;sobel ['fft_fixed']
aes;ann;average;decimation;idct;kasumi;sobel ['snow3g']
fir;interpolation;kasumi;snow3g ['average']
ann;average;fft_fixed;fir;interpolation;qsort ['decimation', 'idct', 'snow3g']
ann;average;decimation;fir;idct;interpolation;kasumi;sobel ['snow3g']
adpcm_encoder;average;kasumi;qsort ['decimation', 'fir', 'idct']
adpcm_encoder;ann;idct;qsort ['fir']
adpcm_encoder;aes;ann;decimation;fft_fixed;fir;kasumi;qsort;snow3g;sobel ['idct']
adpcm_encoder;aes;decimation;interpolation;kasumi;qsort;snow3g ['average', 'fir']
fft_fixed;fir;kasumi;qsort ['adpcm_encoder', 'aes', 'ann', 'average', 'decimation', 'sobel']
adpcm_encoder;average;qsort;snow3g ['a

aes;decimation;idct;interpolation;kasumi;qsort;sobel ['average', 'fir', 'snow3g']
adpcm_encoder;average;decimation;fft_fixed;interpolation;kasumi ['aes', 'idct']
fir;idct;kasumi;qsort;snow3g ['average', 'decimation', 'sobel']
adpcm_encoder;average;decimation;fft_fixed;idct;interpolation;kasumi;qsort;snow3g ['fir', 'sobel']
adpcm_encoder;aes;ann;average;decimation;kasumi ['fir', 'idct']
adpcm_encoder;aes;ann;fir;interpolation;qsort;snow3g ['average']
decimation;fft_fixed;idct;qsort;snow3g ['average', 'sobel']
aes;fir;idct;kasumi;qsort;snow3g;sobel ['average', 'decimation']
aes;decimation;fft_fixed;fir;interpolation;sobel ['average', 'idct']
aes;ann;average;decimation;kasumi;snow3g ['sobel']
adpcm_encoder;aes;ann;fft_fixed;fir;kasumi;snow3g ['average', 'idct']
adpcm_encoder;decimation;idct;interpolation;qsort ['average', 'fft_fixed']
ann;average;decimation;idct;sobel ['fft_fixed']
aes;ann;decimation;fft_fixed;interpolation;snow3g;sobel ['average']
aes;ann;average;decimation;fft_fixed;fir

average;fft_fixed;idct;qsort;snow3g ['decimation', 'sobel']
adpcm_encoder;fir;kasumi ['decimation', 'idct']
aes;decimation;fir;idct;interpolation;kasumi;sobel ['average', 'snow3g']
aes;average;fft_fixed;fir;interpolation;qsort;snow3g;sobel ['decimation', 'idct']
aes;average;decimation;fft_fixed;fir;idct;interpolation;kasumi;snow3g ['sobel']
aes;ann;decimation;fir;kasumi ['average']
adpcm_encoder;aes;average;decimation;fir;interpolation ['sobel']
average;fir;idct;interpolation;kasumi;qsort;snow3g ['decimation']
ann;average;fft_fixed;fir;idct;qsort;snow3g;sobel ['decimation', 'interpolation']
aes;fir;idct;interpolation;qsort;sobel ['average', 'decimation', 'snow3g']
adpcm_encoder;ann;fft_fixed;idct;interpolation ['fir']
aes;decimation;fft_fixed;fir;interpolation;kasumi;qsort;snow3g;sobel ['average', 'idct']
adpcm_encoder;ann;fft_fixed;fir;idct;interpolation;sobel ['decimation']
aes;ann;decimation;fft_fixed;fir;interpolation;kasumi;qsort ['idct']
adpcm_encoder;aes;ann;average;fft_fixed;id

adpcm_encoder;ann;decimation;fft_fixed;kasumi;qsort ['fir', 'idct']
adpcm_encoder;aes;ann;decimation;kasumi;qsort;sobel ['fir', 'idct']
adpcm_encoder;average;decimation;fir;idct;interpolation;qsort;sobel ['fft_fixed']
aes;ann;fft_fixed;idct;sobel ['decimation', 'fir', 'interpolation']
adpcm_encoder;ann;average;fft_fixed;kasumi;qsort;snow3g;sobel ['fir', 'idct']
aes;decimation;interpolation;qsort ['average', 'idct', 'snow3g', 'sobel']
average;decimation;kasumi;qsort;snow3g;sobel ['idct']
ann;average;snow3g ['decimation', 'fir']
adpcm_encoder;aes;average;idct;kasumi;qsort ['decimation', 'sobel']
adpcm_encoder;aes;ann;fft_fixed;idct;kasumi;qsort;snow3g;sobel ['decimation', 'fir', 'interpolation']
ann;fft_fixed;fir;idct;interpolation;snow3g ['decimation']
aes;average;interpolation;snow3g ['fir']
adpcm_encoder;ann;average;decimation;interpolation;qsort;snow3g;sobel ['fir']
aes;average;fir;qsort;snow3g ['idct', 'kasumi']
fft_fixed;idct;kasumi;sobel ['average', 'decimation']
aes;ann;average;d

...

adpcm_encoder;ann;fft_fixed;fir;interpolation;qsort;snow3g ['decimation', 'sobel']
adpcm_encoder;aes;average;idct;kasumi;qsort;sobel ['decimation']
adpcm_encoder;aes;ann;average;fft_fixed;idct;interpolation;kasumi ['decimation', 'fir', 'snow3g']
adpcm_encoder;aes;average;decimation;interpolation ['fir', 'idct', 'snow3g']
ann;decimation;fft_fixed;fir;snow3g;sobel ['idct']
adpcm_encoder;fir;idct;snow3g ['average', 'decimation', 'kasumi']
ann;fft_fixed;fir;idct;qsort;snow3g;sobel ['decimation', 'interpolation']
ann;average;fft_fixed;fir;idct;sobel ['decimation', 'interpolation']
adpcm_encoder;aes;ann;decimation;fft_fixed;snow3g;sobel ['fir', 'idct']
adpcm_encoder;aes;average;interpolation;qsort;snow3g;sobel ['decimation', 'fir']
aes;ann;decimation;fir;idct;interpolation;kasumi;qsort ['snow3g']
adpcm_encoder;aes;average;interpolation;qsort;sobel ['decimation', 'fir', 'idct', 'snow3g']
average;fft_fixed;snow3g ['aes', 'decimation', 'fir', 'idct', 'kasumi']
adpcm_encoder;decimation;fft_fixed

adpcm_encoder;average;decimation;fir;snow3g ['idct']
adpcm_encoder;aes;decimation;fft_fixed;fir;snow3g;sobel ['average', 'idct', 'kasumi']
adpcm_encoder;average;fft_fixed;idct;snow3g ['decimation', 'sobel']
adpcm_encoder;aes;average;fir;interpolation;qsort;sobel ['decimation']
average;fft_fixed;interpolation;qsort ['aes', 'decimation', 'fir', 'idct', 'snow3g', 'sobel']
adpcm_encoder;aes;fir;interpolation;qsort;snow3g ['average', 'decimation']
ann;decimation;fir;idct;kasumi ['snow3g']
average;decimation;interpolation;kasumi;qsort;sobel ['fir']
aes;fir;qsort;snow3g;sobel ['idct']
fir;kasumi;qsort ['idct']
adpcm_encoder;fft_fixed;fir;idct;interpolation;snow3g;sobel ['average']
aes;average;fft_fixed;interpolation;sobel ['decimation', 'fir', 'idct', 'snow3g']
adpcm_encoder;aes;ann;average;fft_fixed;qsort ['decimation', 'sobel']
adpcm_encoder;aes;ann;fir ['decimation']
adpcm_encoder;aes;ann;average;fft_fixed;interpolation;snow3g ['fir', 'idct']
adpcm_encoder;ann;average;fft_fixed;fir;idct;in

adpcm_encoder;ann;fir;interpolation ['idct']
decimation;fir;interpolation;qsort;snow3g ['average']
decimation;fft_fixed;fir;kasumi;qsort;sobel ['aes', 'average', 'idct']
adpcm_encoder;aes;average;fft_fixed;idct;interpolation;kasumi;sobel ['decimation', 'fir', 'snow3g']
adpcm_encoder;ann;fft_fixed;fir;idct;qsort ['decimation', 'interpolation']
adpcm_encoder;ann;kasumi;sobel ['average', 'fir', 'idct']
decimation;idct;kasumi;qsort;snow3g ['average', 'sobel']
adpcm_encoder;aes;average;fft_fixed;idct;kasumi;snow3g ['decimation', 'sobel']
adpcm_encoder;aes;decimation;qsort;snow3g ['average', 'fir', 'idct', 'kasumi']
ann;average;decimation;fft_fixed;interpolation;kasumi ['fir', 'idct']
adpcm_encoder;aes;average;decimation;snow3g ['fir', 'idct']
average;idct;kasumi;qsort;snow3g;sobel ['decimation']
average;fir;kasumi;snow3g ['idct']
adpcm_encoder;ann;average;interpolation;kasumi;qsort;snow3g ['fir']
adpcm_encoder;aes;average;decimation;fft_fixed;interpolation;kasumi;qsort ['idct']
adpcm_encode

aes;decimation;fir;kasumi;qsort;snow3g;sobel ['idct']
adpcm_encoder;aes;fft_fixed;fir;kasumi;snow3g;sobel ['average', 'decimation', 'idct']
ann;kasumi;qsort;sobel ['fir', 'idct']
adpcm_encoder;aes;ann;average;kasumi;snow3g ['fir', 'idct']
average;fir;idct;snow3g ['decimation', 'interpolation', 'kasumi', 'sobel']
adpcm_encoder;average;decimation;fft_fixed;fir;idct ['interpolation', 'kasumi', 'snow3g', 'sobel']
adpcm_encoder;aes;average;fft_fixed;fir;idct;kasumi ['decimation', 'sobel']
adpcm_encoder;ann;average;fir;idct;interpolation;kasumi ['snow3g']
adpcm_encoder;aes;ann;fir;idct;kasumi;qsort ['snow3g']
adpcm_encoder;decimation;fft_fixed;idct;interpolation ['average', 'fir', 'sobel']
aes;fft_fixed;idct;snow3g ['average', 'decimation', 'sobel']
adpcm_encoder;aes;ann;average;fft_fixed;fir;kasumi ['idct']
adpcm_encoder;aes;ann;average;fir;kasumi ['idct']
ann;decimation;fir;interpolation;sobel ['idct']
aes;average;fir;interpolation;kasumi;snow3g;sobel ['decimation']
adpcm_encoder;aes;ann;f

adpcm_encoder;aes;ann;fft_fixed;fir;snow3g ['average', 'idct']
aes;fir;kasumi ['decimation', 'idct']
ann;average;decimation;fft_fixed;idct;interpolation;qsort;sobel ['fir']
adpcm_encoder;aes;average;fft_fixed;idct;snow3g;sobel ['decimation', 'kasumi']
average;decimation;fft_fixed;fir;kasumi;snow3g ['sobel']
adpcm_encoder;aes;fft_fixed;fir;idct;interpolation;kasumi;qsort ['average', 'decimation', 'snow3g', 'sobel']
adpcm_encoder;average;fft_fixed;fir;interpolation;snow3g;sobel ['aes', 'decimation', 'idct']
adpcm_encoder;aes;decimation;fir;idct;interpolation;kasumi ['average', 'snow3g']
aes;fft_fixed;fir;idct;interpolation;kasumi;sobel ['average', 'decimation', 'snow3g']
adpcm_encoder;aes;ann;average;fft_fixed;idct;qsort;sobel ['decimation', 'fir', 'interpolation', 'snow3g']
ann;average;decimation;fft_fixed;fir;interpolation;sobel ['idct', 'snow3g']
adpcm_encoder;aes;decimation;fft_fixed;fir;idct;interpolation;kasumi;snow3g ['average', 'sobel']
adpcm_encoder;fft_fixed;fir;kasumi;sobel ['

In [321]:
es_v4[es_v4['train'] == 'adpcm_encoder;average;decimation;fft_fixed;interpolation']

,train,test,adrs_ave,adrs_max,adrs_ave_rms,adrs_max_rms,hypervolume,dominance,cardinality
12605,adpcm_encoder;average;decimation;fft_fixed;int...,aes,0.0000,0.0000,0.0000,0.0000,1.0000,1.0000,6.0
12606,adpcm_encoder;average;decimation;fft_fixed;int...,ann,0.0175,0.0755,0.0472,0.2295,0.9584,0.4444,7.0
12607,adpcm_encoder;average;decimation;fft_fixed;int...,fir,0.0000,0.0000,0.0000,0.0000,0.4028,1.0000,4.0
12608,adpcm_encoder;average;decimation;fft_fixed;int...,idct,0.0185,0.0504,0.0118,0.0254,0.8697,0.5000,11.0
12609,adpcm_encoder;average;decimation;fft_fixed;int...,kasumi,0.1375,0.2559,0.1399,0.2621,0.6894,0.2857,4.0
12610,adpcm_encoder;average;decimation;fft_fixed;int...,qsort,0.0423,0.1667,0.0164,0.0729,0.7079,0.4286,10.0
12611,adpcm_encoder;average;decimation;fft_fixed;int...,snow3g,0.1039,0.2078,0.0183,0.0366,0.2835,0.5000,8.0
12612,adpcm_encoder;average;decimation;fft_fixed;int...,sobel,0.0000,0.0000,0.0000,0.0000,0.5174,1.0000,2.0


In [322]:
es_v4[es_v4['train'] == 'adpcm_encoder;aes;interpolation;qsort;sobel']

,train,test,adrs_ave,adrs_max,adrs_ave_rms,adrs_max_rms,hypervolume,dominance,cardinality
11581,adpcm_encoder;aes;interpolation;qsort;sobel,ann,4.1159,6.7426,3.7308,6.2494,0.2989,0.1111,4.0
11582,adpcm_encoder;aes;interpolation;qsort;sobel,average,0.0000,0.0000,0.0000,0.0000,1.0000,1.0000,1.0
11583,adpcm_encoder;aes;interpolation;qsort;sobel,decimation,0.0100,0.0551,0.0162,0.0572,0.9895,0.7143,8.0
11584,adpcm_encoder;aes;interpolation;qsort;sobel,fft_fixed,0.2147,0.3722,0.1230,0.2123,0.8123,0.0000,4.0
11585,adpcm_encoder;aes;interpolation;qsort;sobel,fir,0.0000,0.0000,0.0000,0.0000,0.4028,1.0000,4.0
11586,adpcm_encoder;aes;interpolation;qsort;sobel,idct,0.0257,0.0522,0.0281,0.0594,0.8183,0.5000,10.0
11587,adpcm_encoder;aes;interpolation;qsort;sobel,kasumi,0.1375,0.2559,0.1399,0.2621,0.7486,0.2857,3.0
11588,adpcm_encoder;aes;interpolation;qsort;sobel,snow3g,0.1039,0.2078,0.1040,0.2080,0.3073,0.5000,9.0


In [323]:
es_v4[es_v4['train'] == 'adpcm_encoder;decimation;idct;kasumi']

,train,test,adrs_ave,adrs_max,adrs_ave_rms,adrs_max_rms,hypervolume,dominance,cardinality
5224,adpcm_encoder;decimation;idct;kasumi,aes,0.0386,0.1712,0.1054,0.3564,0.9801,0.6000,4.0
5225,adpcm_encoder;decimation;idct;kasumi,ann,0.2880,0.5552,0.3263,0.8176,0.5840,0.2222,5.0
5226,adpcm_encoder;decimation;idct;kasumi,average,0.0000,0.0000,0.0000,0.0000,1.0000,1.0000,1.0
5227,adpcm_encoder;decimation;idct;kasumi,fft_fixed,0.9692,1.2886,0.9692,1.2886,0.5282,0.0000,1.0
5228,adpcm_encoder;decimation;idct;kasumi,fir,0.0000,0.0000,0.0000,0.0000,0.5632,1.0000,5.0
5229,adpcm_encoder;decimation;idct;kasumi,interpolation,0.0956,0.2351,0.0969,0.2430,0.5948,0.4286,12.0
5230,adpcm_encoder;decimation;idct;kasumi,qsort,0.5559,1.3749,0.5558,1.3745,0.4601,0.2857,4.0
5231,adpcm_encoder;decimation;idct;kasumi,snow3g,0.1039,0.2078,0.1040,0.2080,0.2477,0.5000,7.0
5232,adpcm_encoder;decimation;idct;kasumi,sobel,0.0000,0.0000,0.0000,0.0000,0.4468,1.0000,5.0


In [316]:
dm_v4

,benchmark,adrs_ave,adrs_max,adrs_ave_rms,adrs_max_rms,hypervolume,dominance,cardinality
0,adpcm_encoder,0.0251,0.0978,0.0261,0.0875,0.2722,0.6667,6.0
1,aes,0.0043,0.0216,0.0319,0.1593,0.9669,0.8000,6.0
2,ann,0.0142,0.0600,0.0891,0.5433,0.9356,0.5556,8.0
3,average,0.0000,0.0000,0.0000,0.0000,0.4526,1.0000,8.0
4,decimation,0.0541,0.1482,0.0404,0.1479,0.8890,0.2857,4.0
5,fft_fixed,0.0583,0.1154,0.0383,0.0856,0.7895,0.0000,8.0
6,fir,0.0000,0.0000,0.0000,0.0000,0.3377,1.0000,5.0
7,idct,0.0257,0.0522,0.0281,0.0594,0.7248,0.5000,9.0
8,interpolation,0.2575,0.3985,0.1646,0.2454,0.5816,0.1429,15.0
9,kasumi,0.1375,0.2559,0.1399,0.2621,0.7833,0.2857,4.0


Show the benchmarks, which form the training set, leading better results than direct mapping with tolerance on the rest testing benchmarks.

In [317]:
train_combos = set(es_v4['train'])
for combo in train_combos:
    combo_list = combo.split(';')
    metrics_1 = ['adrs_ave']
    metrics_2 = ['hypervolume', 'dominance']
    bool_1 = es_v4[es_v4['train'] == combo][metrics_1].reset_index(drop=True) <= \
             dm_v4_01[~dm_v4_01['benchmark'].isin(combo_list)][metrics_1].reset_index(drop=True)
    bool_2 = es_v4[es_v4['train'] == combo][metrics_2].reset_index(drop=True) >= \
             dm_v4_01[~dm_v4_01['benchmark'].isin(combo_list)][metrics_2].reset_index(drop=True)
    bool_all = pd.concat([bool_1, bool_2], axis=1)
    if np.all(bool_all):
        print(combo)

adpcm_encoder;aes;ann;decimation;fft_fixed;idct;interpolation;qsort;snow3g;sobel
adpcm_encoder;aes;ann;decimation;fft_fixed;idct;interpolation;qsort;snow3g
adpcm_encoder;aes;ann;decimation;fft_fixed;idct;interpolation;kasumi;qsort;snow3g;sobel
adpcm_encoder;aes;ann;decimation;fft_fixed;fir;idct;interpolation;qsort;snow3g
adpcm_encoder;aes;ann;average;decimation;fft_fixed;idct;interpolation;qsort;snow3g;sobel
adpcm_encoder;aes;ann;decimation;fft_fixed;fir;idct;interpolation;kasumi;qsort;snow3g;sobel
adpcm_encoder;aes;ann;average;decimation;fft_fixed;fir;idct;interpolation;qsort;snow3g
adpcm_encoder;aes;ann;decimation;fft_fixed;fir;idct;interpolation;kasumi;qsort;snow3g
adpcm_encoder;aes;ann;average;decimation;fft_fixed;fir;idct;interpolation;kasumi;qsort;snow3g
adpcm_encoder;aes;ann;average;decimation;fft_fixed;idct;interpolation;kasumi;qsort;snow3g;sobel
adpcm_encoder;aes;ann;decimation;fft_fixed;fir;idct;interpolation;qsort;snow3g;sobel
adpcm_encoder;aes;ann;decimation;fft_fixed;idct;

In [318]:
es_v4[es_v4['train'] == 'adpcm_encoder;aes;ann;decimation;fft_fixed;idct;interpolation;qsort;snow3g']

,train,test,adrs_ave,adrs_max,adrs_ave_rms,adrs_max_rms,hypervolume,dominance,cardinality
49944,adpcm_encoder;aes;ann;decimation;fft_fixed;idc...,average,0.0000,0.0000,0.0000,0.0000,0.3966,1.0000,2.0
49945,adpcm_encoder;aes;ann;decimation;fft_fixed;idc...,fir,0.0000,0.0000,0.0000,0.0000,0.6664,1.0000,4.0
49946,adpcm_encoder;aes;ann;decimation;fft_fixed;idc...,kasumi,0.1375,0.2559,0.1399,0.2621,0.7486,0.2857,3.0
49947,adpcm_encoder;aes;ann;decimation;fft_fixed;idc...,sobel,0.0000,0.0000,0.0000,0.0000,0.2347,1.0000,10.0


In [319]:
dm_v4_01

,benchmark,adrs_ave,adrs_max,adrs_ave_rms,adrs_max_rms,hypervolume,dominance,cardinality
0,adpcm_encoder,0.0000,0.0000,0.0000,0.0000,0.3447,1.0000,21.0
1,aes,0.0000,0.0000,0.0000,0.0000,0.6741,1.0000,10.0
2,ann,0.0075,0.0488,0.0082,0.0369,0.3983,0.6667,208.0
3,average,0.0000,0.0000,0.0000,0.0000,0.2961,1.0000,14.0
4,decimation,0.0100,0.0551,0.0124,0.0440,0.9872,0.7143,10.0
5,fft_fixed,0.0000,0.0000,0.0000,0.0000,0.5822,1.0000,250.0
6,fir,0.0000,0.0000,0.0000,0.0000,0.1056,1.0000,47.0
7,idct,0.0126,0.0504,0.0031,0.0122,0.2512,0.7500,217.0
8,interpolation,0.0000,0.0000,0.0000,0.0000,0.3620,1.0000,1623.0
9,kasumi,0.1375,0.2559,0.1399,0.2621,0.5929,0.2857,25.0


Similarity measuring

In [225]:
benchmarks = list(dm_v4['benchmark'])
metrics_moo = list(dm_v4)[1:]
metrics_moo

['adrs_ave',
 'adrs_max',
 'adrs_ave_rms',
 'adrs_max_rms',
 'hypervolume',
 'dominance',
 'cardinality']

In [226]:
similarity = dict()
for benchmark in benchmarks:
    similarity[benchmark] = dict(zip(benchmarks, [0] * len(benchmarks)))
    dm_metrics = dm_v4[dm_v4['benchmark'].str.contains(benchmark)][metrics_moo[:-1]].values
    es_v4_benchmark = es_v4[es_v4['test'].str.contains(benchmark)]
    adrs_bool = es_v4_benchmark[metrics_moo[:4]].values <= dm_metrics[:, :4]
    others_bool = es_v4_benchmark[metrics_moo[4:-1]].values >= dm_metrics[:, 4:]
    adrs_bool = np.all(adrs_bool, axis=1)
    others_bool = np.all(others_bool, axis=1)
    final_bool = np.all(np.c_[adrs_bool, others_bool], axis=1)
    good_combinations = es_v4_benchmark[final_bool]
    for combination in good_combinations['train']:
        combo_list = combination.split(';')
        for k in combo_list:
            similarity[benchmark][k] += 1

In [227]:
simi_mat = pd.DataFrame(0, index=benchmarks, columns=benchmarks)
simi_mat

,adpcm_encoder,aes,ann,average,decimation,fft_fixed,fir,idct,interpolation,kasumi,qsort,snow3g,sobel
adpcm_encoder,0,0,0,0,0,0,0,0,0,0,0,0,0
aes,0,0,0,0,0,0,0,0,0,0,0,0,0
ann,0,0,0,0,0,0,0,0,0,0,0,0,0
average,0,0,0,0,0,0,0,0,0,0,0,0,0
decimation,0,0,0,0,0,0,0,0,0,0,0,0,0
fft_fixed,0,0,0,0,0,0,0,0,0,0,0,0,0
fir,0,0,0,0,0,0,0,0,0,0,0,0,0
idct,0,0,0,0,0,0,0,0,0,0,0,0,0
interpolation,0,0,0,0,0,0,0,0,0,0,0,0,0
kasumi,0,0,0,0,0,0,0,0,0,0,0,0,0


In [229]:
for target, sources in similarity.items():
    for benchmark, count in sources.items():
        simi_mat.loc[benchmark, target] = count

simi_mat

,adpcm_encoder,aes,ann,average,decimation,fft_fixed,fir,idct,interpolation,kasumi,qsort,snow3g,sobel
adpcm_encoder,0,266,42,1214,1044,20,1323,1364,351,275,3,167,452
aes,39,0,44,1187,1075,4,1327,1332,352,277,8,151,454
ann,9,0,0,617,656,0,1506,1385,532,0,0,68,141
average,27,258,48,0,1063,21,1290,1315,363,255,5,161,505
decimation,35,267,54,1192,0,21,1305,1339,383,270,9,153,481
fft_fixed,63,435,96,1219,1526,0,1447,1597,539,338,16,324,677
fir,33,264,56,1194,1048,17,0,1306,354,254,6,154,481
idct,0,1,0,1095,1183,27,1089,0,566,264,0,198,712
interpolation,6,255,18,1312,1092,24,1388,1001,0,10,0,252,448
kasumi,44,207,65,1215,1037,0,1240,1261,271,0,16,116,466


In [232]:
simi_mat_t = simi_mat.transpose()
simi_space = simi_mat + simi_mat_t
simi_space

,adpcm_encoder,aes,ann,average,decimation,fft_fixed,fir,idct,interpolation,kasumi,qsort,snow3g,sobel
adpcm_encoder,0,305,51,1241,1079,83,1356,1364,357,319,26,173,472
aes,305,0,44,1445,1342,439,1591,1333,607,484,272,361,715
ann,51,44,0,665,710,96,1562,1385,550,65,37,68,176
average,1241,1445,665,0,2255,1240,2484,2410,1675,1470,1191,1495,1649
decimation,1079,1342,710,2255,0,1547,2353,2522,1475,1307,1157,1120,1544
fft_fixed,83,439,96,1240,1547,0,1464,1624,563,338,20,330,697
fir,1356,1591,1562,2484,2353,1464,0,2395,1742,1494,1360,1401,1787
idct,1364,1333,1385,2410,2522,1624,2395,0,1567,1525,1337,1496,2017
interpolation,357,607,550,1675,1475,563,1742,1567,0,281,234,526,837
kasumi,319,484,65,1470,1307,338,1494,1525,281,0,275,394,696


Clustering

In [261]:
import scipy.cluster.hierarchy as hcluster
thresh = [500, 600, 700, 800, 900, 1000, 1500, 2000, 2500, 3000, 3500, 4000, 4500, 5000]
for i in thresh:
    clusters = hcluster.fclusterdata(simi_space.as_matrix(), i, criterion='distance')
    print(i, clusters)

500 [ 1  3  7 10  9  4 11 12  6  5  1  2  8]
600 [ 1  2  6  9  8  3 10 11  5  4  1  1  7]
700 [1 1 3 6 5 1 7 8 2 1 1 1 4]
800 [1 1 2 5 4 1 6 7 1 1 1 1 3]
900 [1 1 1 4 3 1 5 6 1 1 1 1 2]
1000 [1 1 1 4 3 1 5 6 1 1 1 1 2]
1500 [1 1 1 3 2 1 4 5 1 1 1 1 1]
2000 [1 1 1 3 2 1 4 5 1 1 1 1 1]
2500 [1 1 1 3 2 1 4 5 1 1 1 1 1]
3000 [1 1 1 2 1 1 3 4 1 1 1 1 1]
3500 [1 1 1 1 1 1 2 2 1 1 1 1 1]
4000 [1 1 1 1 1 1 1 1 1 1 1 1 1]
4500 [1 1 1 1 1 1 1 1 1 1 1 1 1]
5000 [1 1 1 1 1 1 1 1 1 1 1 1 1]


In [246]:
kmeans_model = KMeans(n_clusters=3, random_state=42).fit(simi_space.as_matrix())
labels = kmeans_model.labels_
labels

array([1, 1, 1, 2, 2, 1, 0, 0, 1, 1, 1, 1, 1])

In [271]:
agglo_model = cluster.AgglomerativeClustering(n_clusters=3, affinity='l2', linkage='average').fit(simi_space.as_matrix())
labels = agglo_model.labels_
labels

C:\Users\sxl131830\AppData\Local\Continuum\Anaconda3\lib\site-packages\sklearn\cluster\hierarchical.py:410: ClusterWarning: scipy.cluster: The symmetric non-negative hollow observation matrix looks suspiciously like an uncondensed distance matrix
  out = hierarchy.linkage(X, method=linkage, metric=affinity)


array([1, 1, 1, 2, 2, 1, 0, 0, 1, 1, 1, 1, 1], dtype=int64)

In [270]:
for n in range(2, len(benchmarks)):
    kmeans_model = cluster.KMeans(n_clusters=n).fit(simi_space.as_matrix())
    labels = kmeans_model.labels_
    print(n, metrics.calinski_harabaz_score(simi_space.as_matrix(), labels))

2 14.9873097652
3 12.1109521027
4 12.9169269731
5 19.0279425235
6 22.2435092842
7 24.5106333233
8 30.4663904613
9 33.3632143702
10 36.0537329965
11 47.0562802298
12 268.578628265


In [266]:
for n in range(2, len(benchmarks)):
    agglo_model = cluster.AgglomerativeClustering(n_clusters=n).fit(simi_space.as_matrix())
    labels = agglo_model.labels_
    print(n, metrics.calinski_harabaz_score(simi_space.as_matrix(), labels))

2 14.9873097652
3 12.1109521027
4 12.9169269731
5 19.0279425235
6 21.5079686292
7 24.3940013778
8 30.4663904613
9 31.8522671363
10 36.0537329965
11 47.0562802298
12 268.578628265


C:\Users\sxl131830\AppData\Local\Continuum\Anaconda3\lib\site-packages\scipy\cluster\hierarchy.py:471: ClusterWarning: scipy.cluster: The symmetric non-negative hollow observation matrix looks suspiciously like an uncondensed distance matrix
  return linkage(y, method='ward', metric='euclidean')


For a benchmark as testing, find the benchmarks as training set that leads to better results than direct mapping.

In [272]:
benchmark = benchmarks[2]

dm_metrics = dm_v4[dm_v4['benchmark'].str.contains(benchmark)][metrics_moo[:-1]].values

es_v4_benchmark = es_v4[es_v4['test'].str.contains(benchmark)]
adrs_bool = es_v4_benchmark[metrics_moo[:4]].values <= dm_metrics[:, :4]
others_bool = es_v4_benchmark[metrics_moo[4:-1]].values >= dm_metrics[:, 4:]

adrs_bool = np.all(adrs_bool, axis=1)

others_bool = np.all(others_bool, axis=1)

final_bool = np.all(np.c_[adrs_bool, others_bool], axis=1)

In [273]:
dm_v4[dm_v4['benchmark'] == benchmark]

,benchmark,adrs_ave,adrs_max,adrs_ave_rms,adrs_max_rms,hypervolume,dominance,cardinality
2,ann,0.0142,0.06,0.0891,0.5433,0.9356,0.5556,8.0


In [274]:
es_v4_benchmark[final_bool]

,train,test,adrs_ave,adrs_max,adrs_ave_rms,adrs_max_rms,hypervolume,dominance,cardinality
377,aes;qsort,ann,0.0075,0.0488,0.0878,0.5433,0.9520,0.6667,7.0
730,fft_fixed;interpolation,ann,0.0142,0.0600,0.0420,0.2295,0.9478,0.5556,8.0
1034,adpcm_encoder;aes;decimation,ann,0.0075,0.0488,0.0878,0.5433,0.9520,0.6667,7.0
1455,adpcm_encoder;fft_fixed;sobel,ann,0.0142,0.0600,0.0420,0.2295,0.9478,0.5556,8.0
2776,average;fft_fixed;interpolation,ann,0.0142,0.0600,0.0420,0.2295,0.9478,0.5556,8.0
3326,fft_fixed;fir;interpolation,ann,0.0142,0.0600,0.0437,0.2295,0.9475,0.5556,7.0
3336,fft_fixed;fir;kasumi,ann,0.0075,0.0488,0.0814,0.5433,0.9529,0.6667,9.0
3456,fft_fixed;interpolation;sobel,ann,0.0142,0.0600,0.0420,0.2295,0.9478,0.5556,8.0
3466,fft_fixed;kasumi;qsort,ann,0.0075,0.0488,0.0814,0.5433,0.9529,0.6667,9.0
3486,fft_fixed;kasumi;sobel,ann,0.0075,0.0488,0.0814,0.5433,0.9529,0.6667,9.0


With tolerance

In [159]:
benchmark = benchmarks[2]

dm_metrics = dm_v4_01[dm_v4_01['benchmark'].str.contains(benchmark)][metrics_moo[:-1]].values

es_v4_benchmark = es_v4[es_v4['test'].str.contains(benchmark)]
adrs_bool = es_v4_benchmark[metrics_moo[:4]].values <= dm_metrics[:, :4]
others_bool = es_v4_benchmark[metrics_moo[4:-1]].values >= dm_metrics[:, 4:]

adrs_bool = np.all(adrs_bool, axis=1)

others_bool = np.all(others_bool, axis=1)

final_bool = np.all(np.c_[adrs_bool, others_bool], axis=1)

In [156]:
dm_v4_01[dm_v4_01['benchmark'] == benchmark]

,benchmark,adrs_ave,adrs_max,adrs_ave_rms,adrs_max_rms,hypervolume,dominance,cardinality
12,sobel,0.0,0.0,0.0,0.0,0.1945,1.0,10.0


In [157]:
es_v4_benchmark[final_bool]

,train,test,adrs_ave,adrs_max,adrs_ave_rms,adrs_max_rms,hypervolume,dominance,cardinality
11,adpcm_encoder,sobel,0.0,0.0,0.0,0.0,0.2056,1.0,11.0
23,aes,sobel,0.0,0.0,0.0,0.0,1.0000,1.0,1.0
35,ann,sobel,0.0,0.0,0.0,0.0,0.6479,1.0,5.0
47,average,sobel,0.0,0.0,0.0,0.0,0.6192,1.0,2.0
59,decimation,sobel,0.0,0.0,0.0,0.0,0.3073,1.0,9.0
71,fft_fixed,sobel,0.0,0.0,0.0,0.0,1.0000,1.0,1.0
83,fir,sobel,0.0,0.0,0.0,0.0,0.3376,1.0,3.0
95,idct,sobel,0.0,0.0,0.0,0.0,0.3024,1.0,8.0
107,interpolation,sobel,0.0,0.0,0.0,0.0,1.0000,1.0,1.0
119,kasumi,sobel,0.0,0.0,0.0,0.0,0.2862,1.0,11.0
